In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from io import StringIO
import os
import time
import requests
from selenium.webdriver.chrome.options import Options

from datetime import date, timedelta
from IPython.core.display import HTML

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [2]:
# !pip install openpyxl
# !pip install pandas
# !pip install numpy

In [3]:
# # Install Selenium
# !pip install selenium

# # Install Webdriver Manager
# !pip install webdriver-manager

# # Check Selenium version
# import selenium
# selenium.__version__


# # Optionally, update Selenium to the latest version
# !pip install -U selenium

In [4]:
url = 'https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B'

driver  = webdriver.Chrome()        
driver.get( url )

In [5]:
driver.maximize_window()

In [6]:
#periodo //*[@id="ctl00_cphContent_rdpDate_dateInput"]


In [7]:
#consulta //*[@id="ctl00_cphContent_btnConsultar"]
consultar_btn = driver.find_element( By.XPATH,  '//*[@id="ctl00_cphContent_btnConsultar"]' )
consultar_btn.click()

In [8]:
download_button = driver.find_element( By.XPATH,  '//*[@id="ctl00_cphContent_btnExportar"]' )
download_button.click()


In [9]:
moneda = 1 #1 nacional 0 extrnajera

if moneda ==1:
    moneda_btn = driver.find_element( By.XPATH,  '//*[@id="ctl00_cphContent_lbtnMn"]' )
    print('Moneda Nacional')
else:
    moneda_btn = driver.find_element( By.XPATH,  '//*[@id="ctl00_cphContent_lbtnMex"]' )
    print('Moneda Extranjera')

moneda_btn.click()

Moneda Nacional


In [10]:
data = driver.find_element(By.XPATH, '//*[@id="ctl00_cphContent_contenidoActualMex"]')
data_tab = data.get_attribute( 'outerHTML' )
table_html_io = StringIO( data_tab )
table_df      = pd.read_html( table_html_io )[ 0 ]
table_df.head(5)

,BBVA,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio,Unnamed: 18
0,Tasa Anual (%),BBVABancomCréditoPichinchaBIFScotiabankCitiban...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BBVA,BBVABancomCréditoPichinchaBIFScotiabankCitiban...,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio
2,6.86,BBVABancomCréditoPichinchaBIFScotiabankCitiban...,-,7.15,7.47,7.75,6.98,-,7.44,-,6.32,-,10.62,-,-,7.03,s.i.,7.13,7.53
3,7.64,BBVABancomCréditoPichinchaBIFScotiabankCitiban...,-,7.58,7.40,6.99,6.80,-,7.26,-,-,-,10.75,-,-,-,s.i.,-,7.67
4,7.73,BBVABancomCréditoPichinchaBIFScotiabankCitiban...,-,6.83,7.35,7.66,6.65,-,6.85,-,5.94,-,10.43,-,-,-,s.i.,7.43,7.03


In [11]:
path_static = '../data/SBS/tasas'
os.makedirs( path_static, exist_ok = True )

path_table = path_static + '/tasas.xlsx'
table_df.to_excel( path_table, index = False )

# Parte de la data - Bancos

In [12]:
# encabezado    //*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[1]
header = driver.find_element(By.XPATH, '//*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[1]')
header = header.get_attribute( 'outerHTML' )
header = StringIO( header )
table_header      = pd.read_html( header )[ 0 ]
table_header.head(5)

,BBVA,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio
0,7.91,-,7.95,8.97,7.88,7.87,9.34,8.42,-,8.76,-,11.01,-,-,10.62,s.i.,7.97,8.10
1,9.06,-,8.88,10.57,5.34,8.40,-,9.08,-,-,-,11.03,-,-,-,s.i.,-,8.71
2,8.25,-,8.02,11.75,8.94,8.54,-,10.80,-,8.76,-,11.78,-,-,-,s.i.,-,8.29
3,8.91,-,8.16,-,9.13,7.88,-,7.73,-,-,-,10.73,-,-,9.29,s.i.,-,8.25
4,8.58,-,8.32,10.30,8.80,8.03,9.34,8.75,-,-,-,12.72,-,-,12.30,s.i.,8.81,8.50


# Fecha

In [13]:
date = date(2023,1,10)
data_date = [date.strftime("%d/%m/%Y")]

btn_fecha = driver.find_element('xpath', '//*[@id="ctl00_cphContent_rdpDate_dateInput"]')
btn_fecha.clear()
btn_fecha.send_keys(data_date)

In [14]:
btn_consulta = driver.find_element('xpath', '//*[@id="ctl00_cphContent_btnConsultar"]')
btn_consulta.click()

# Tipo de tasas

In [15]:
header = driver.find_element(By.XPATH, '//*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[' + str(47) + ']/td')
header = header.get_attribute( 'outerHTML' )
print(header)

# //*[@id="ctl00_cphContent_rpgActualMn_OT__fhRow"]/td

# //*[@id="ctl00_cphContent_rpgActualMn_OT__fhRow"]/td

# //*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[5]/td

# //*[@id="ctl00_cphContent_rpgActualMn_OT__lhRow"]/td

<td class="rpgRowHeaderField rpgRowHeader APLI_fila1_new">   Préstamos hipotecarios para vivienda</td>


In [16]:
#Forma 1

TasaAnual = []

for i in range(1,48):
    header = driver.find_element(By.XPATH, '//*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[' + str(i) + ']/td')
    header = header.get_attribute( 'outerHTML' )
    
    #extraer el texto
    soup = BeautifulSoup(header, 'html.parser')
    text = soup.get_text(strip=True)

    TasaAnual.append(text)
    #print(text)
    

TasaAnual_1 = pd.DataFrame({TasaAnual[0]: TasaAnual[1:]})
# print(TasaAnual_1.head(5))

Data = pd.concat([TasaAnual_1, table_header], axis=1)
Data.head(5)

,Tasa Anual (%),BBVA,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio
0,Corporativos,7.91,-,7.95,8.97,7.88,7.87,9.34,8.42,-,8.76,-,11.01,-,-,10.62,s.i.,7.97,8.10
1,Descuentos,9.06,-,8.88,10.57,5.34,8.40,-,9.08,-,-,-,11.03,-,-,-,s.i.,-,8.71
2,Préstamos hasta 30 días,8.25,-,8.02,11.75,8.94,8.54,-,10.80,-,8.76,-,11.78,-,-,-,s.i.,-,8.29
3,Préstamos de 31 a 90 días,8.91,-,8.16,-,9.13,7.88,-,7.73,-,-,-,10.73,-,-,9.29,s.i.,-,8.25
4,Préstamos de 91 a 180 días,8.58,-,8.32,10.30,8.80,8.03,9.34,8.75,-,-,-,12.72,-,-,12.30,s.i.,8.81,8.50


In [17]:
Data = pd.concat([TasaAnual_1, table_header], axis=1)
Data.head(5)

,Tasa Anual (%),BBVA,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio
0,Corporativos,7.91,-,7.95,8.97,7.88,7.87,9.34,8.42,-,8.76,-,11.01,-,-,10.62,s.i.,7.97,8.10
1,Descuentos,9.06,-,8.88,10.57,5.34,8.40,-,9.08,-,-,-,11.03,-,-,-,s.i.,-,8.71
2,Préstamos hasta 30 días,8.25,-,8.02,11.75,8.94,8.54,-,10.80,-,8.76,-,11.78,-,-,-,s.i.,-,8.29
3,Préstamos de 31 a 90 días,8.91,-,8.16,-,9.13,7.88,-,7.73,-,-,-,10.73,-,-,9.29,s.i.,-,8.25
4,Préstamos de 91 a 180 días,8.58,-,8.32,10.30,8.80,8.03,9.34,8.75,-,-,-,12.72,-,-,12.30,s.i.,8.81,8.50


In [20]:
path_static = '../data/SBS/tasas_final'
os.makedirs( path_static, exist_ok = True )

path_table = path_static + '/tasas_final_'+date.strftime("%d.%m.%Y")+'.xlsx'
Data.to_excel( path_table, index = False )

In [21]:
#Forma 2
TasaAnual = []
for i in range(2,48):
    header = driver.find_element(By.XPATH, '//*[@id="ctl00_cphContent_rpgActualMn_OT"]/tbody/tr[' + str(i) + ']/td')
    header = header.get_attribute( 'outerHTML' )
    
    #extraer el texto
    soup = BeautifulSoup(header, 'html.parser')
    text = soup.get_text(strip=True)

    TasaAnual.append(text)
    #print(text)

TasaAnual = pd.DataFrame(TasaAnual)   
TasaAnual = TasaAnual.rename(columns={0: 'Tasa Anual %'})


Data_Final = pd.concat([TasaAnual, table_header], axis=1)
Data_Final.head(5)

,Tasa Anual %,BBVA,Bancom,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,BCI,Promedio
0,Corporativos,7.91,-,7.95,8.97,7.88,7.87,9.34,8.42,-,8.76,-,11.01,-,-,10.62,s.i.,7.97,8.10
1,Descuentos,9.06,-,8.88,10.57,5.34,8.40,-,9.08,-,-,-,11.03,-,-,-,s.i.,-,8.71
2,Préstamos hasta 30 días,8.25,-,8.02,11.75,8.94,8.54,-,10.80,-,8.76,-,11.78,-,-,-,s.i.,-,8.29
3,Préstamos de 31 a 90 días,8.91,-,8.16,-,9.13,7.88,-,7.73,-,-,-,10.73,-,-,9.29,s.i.,-,8.25
4,Préstamos de 91 a 180 días,8.58,-,8.32,10.30,8.80,8.03,9.34,8.75,-,-,-,12.72,-,-,12.30,s.i.,8.81,8.50


# Data a Excel

In [ ]:
path_static = '../data/SBS/tasas_final'
os.makedirs( path_static, exist_ok = True )

path_table = path_static + '/tasas_final_'+date.strftime("%d.%m.%Y")+'.xlsx'
Data_Final.to_excel( path_table, index = False )